In [3]:
'''
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)


# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)


'''

'\n# Dependencies and Setup\nimport matplotlib.pyplot as plt\nimport pandas as pd\nimport numpy as np\nimport requests\nimport time\nfrom scipy.stats import linregress\n\n# Import API key\nfrom api_keys import weather_api_key\n\n# Incorporated citipy to determine city based on latitude and longitude\nfrom citipy import citipy\n\n# Output File (CSV)\noutput_data_file = "output_data/cities.csv"\n\n# Range of latitudes and longitudes\nlat_range = (-90, 90)\nlng_range = (-180, 180)\n\n\n# List for holding lat_lngs and cities\nlat_lngs = []\ncities = []\n\n# Create a set of random lat and lng combinations\nlats = np.random.uniform(lat_range[0], lat_range[1], size=1500)\nlngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)\nlat_lngs = zip(lats, lngs)\n\n# Identify nearest city for each lat, lng combination\nfor lat_lng in lat_lngs:\n    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name\n    \n    # If the city is unique, then add it to a our cities list\n    if city not

In [4]:
# Import utilities
import pandas as pd
import requests
import json
import time
import math
from pprint import pprint

In [5]:
# Import charge station cities list
chargeStationCitiesdf = pd.read_csv("alt_fuel_stations_city_count.csv", encoding = "UTF-8", index_col = 0)
chargeStationCitiesdf.head(3)

,No. of Charge Stations
Unique City,
"ABBOTSFORD, WI",1
"ABERDEEN, MD",6
"ABERDEEN, NJ",1


In [6]:
# Import cities list with population density and coordinates and clean
uscitiesdf = pd.read_csv("uscities.csv", encoding = "UTF-8", index_col = 0)
uscitiesdf.head(3)

,city_ascii,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
city,,,,,,,,,,,,,,,,
New York,New York,NY,New York,36061,New York,40.6943,-73.9249,18713220,10715.0,polygon,False,True,America/New_York,1,11229 11226 11225 11224 11222 11221 11220 1138...,1840034016
Los Angeles,Los Angeles,CA,California,6037,Los Angeles,34.1139,-118.4068,12750807,3276.0,polygon,False,True,America/Los_Angeles,1,90291 90293 90292 91316 91311 90037 90031 9000...,1840020491
Chicago,Chicago,IL,Illinois,17031,Cook,41.8373,-87.6862,8604203,4574.0,polygon,False,True,America/Chicago,1,60018 60649 60641 60640 60643 60642 60645 6064...,1840000494


In [7]:
#pull columns city_ascii, state_id, lat, lng
uscitiesColRemoved = uscitiesdf[["city_ascii","state_id","state_name","lat","lng","population","density"]]
# capitalize city_ascii
uscitiesColRemoved["city_ascii"] = uscitiesColRemoved["city_ascii"].str.upper()
# rename density column for clarification
uscitiesClean = uscitiesColRemoved.rename(columns = {"density":"Density(km^2)"}, copy = True)
# create unique city column for merging
uscitiesClean["Unique City"] = uscitiesClean["city_ascii"] + ", " + uscitiesClean["state_id"]
uscitiesClean

C:\Users\sanab\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,city_ascii,state_id,state_name,lat,lng,population,Density(km^2),Unique City
city,,,,,,,,
New York,NEW YORK,NY,New York,40.6943,-73.9249,18713220,10715.0,"NEW YORK, NY"
Los Angeles,LOS ANGELES,CA,California,34.1139,-118.4068,12750807,3276.0,"LOS ANGELES, CA"
Chicago,CHICAGO,IL,Illinois,41.8373,-87.6862,8604203,4574.0,"CHICAGO, IL"
Miami,MIAMI,FL,Florida,25.7839,-80.2102,6445545,5019.0,"MIAMI, FL"
Dallas,DALLAS,TX,Texas,32.7936,-96.7662,5743938,1526.0,"DALLAS, TX"
...,...,...,...,...,...,...,...,...
Poso Park,POSO PARK,CA,California,35.8114,-118.6364,2,17.0,"POSO PARK, CA"
Lotsee,LOTSEE,OK,Oklahoma,36.1334,-96.2091,2,39.0,"LOTSEE, OK"
The Ranch,THE RANCH,MN,Minnesota,47.3198,-95.6952,2,2.0,"THE RANCH, MN"


In [8]:
# import gas prices per CA city and clean
CAgaspricesdf = pd.read_csv("CA_gas_prices_by_city.csv", encoding = "UTF-8", index_col = 0)
# re-index to pull city names into their own series
CAgaspricesIndexed = CAgaspricesdf.reset_index(drop = False)
# capitalize City
CAgaspricesIndexed["City"] = CAgaspricesIndexed["City"].str.upper()
# add "CA" to match with unique cities in cities dataframe
CAgaspricesIndexed["Unique City"] = CAgaspricesIndexed["City"] + ", CA"
CAgaspricesIndexed

,City,Price,Unique City
0,BAKERSFIELD,$3.19,"BAKERSFIELD, CA"
1,CHICO-PARADISE,$3.19,"CHICO-PARADISE, CA"
2,EL CENTRO,$3.19,"EL CENTRO, CA"
3,FRESNO,$3.20,"FRESNO, CA"
4,HANFORD,$3.13,"HANFORD, CA"
5,CORCORAN,$3.13,"CORCORAN, CA"
6,LOS ANGELES,$3.28,"LOS ANGELES, CA"
7,LONG BEACH,$3.28,"LONG BEACH, CA"
8,MADERA,$3.18,"MADERA, CA"
9,CHOWCHILLA,$3.18,"CHOWCHILLA, CA"


In [9]:
# IS THIS NECESSARY??? Change to just simple counts for yelp responses?

# import city landmass list and clean
city_landmass = pd.read_csv("city_landmass.csv", encoding = "ISO-8859-1", index_col = 0)
#pull columns city_ascii, state_id, lat, lng
city_landmassColRemoved = city_landmass[["Unique City","Land Area (sq km)"]]
# change "Land Area (sq km)" to float
city_landmassColRemoved["Land Area (sq km)"] = city_landmassColRemoved["Land Area (sq km)"].str.replace(",","").astype(float)
city_landmassColRemoved["Land Area (sq km)"] = city_landmassColRemoved["Land Area (sq km)"].astype(int)
city_landmassColRemoved

C:\Users\sanab\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sanab\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,Unique City,Land Area (sq km)
Rank,,
1,"NEW YORK, NY",8936
2,"LOS ANGELES, CA",4496
3,"CHICAGO, IL",6326
4,"MIAMI, FL",3208
5,"PHILADELPHIA, PA",5131
...,...,...
493,"CHAMBERSBURG, PA",96
494,"GRANTS PASS, OR",70
495,"NEW BERN, NC",112


In [10]:
#calculate radius for Land Area
radiusList = []
for index, row in city_landmassColRemoved.iterrows():
    radiusList.append(math.sqrt(row["Land Area (sq km)"]/math.pi)*1000)
city_landmassColRemoved["radius m"] = radiusList
city_landmassColRemoved["radius m"] = city_landmassColRemoved["radius m"].astype(int)
city_landmassColRemoved

C:\Users\sanab\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\sanab\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unique City,Land Area (sq km),radius m
Rank,,,
1,"NEW YORK, NY",8936,53333
2,"LOS ANGELES, CA",4496,37830
3,"CHICAGO, IL",6326,44873
4,"MIAMI, FL",3208,31955
5,"PHILADELPHIA, PA",5131,40413
...,...,...,...
493,"CHAMBERSBURG, PA",96,5527
494,"GRANTS PASS, OR",70,4720
495,"NEW BERN, NC",112,5970


In [11]:
# merge station counts to the cities df as an inner merge
uscitiesStations = pd.merge(uscitiesClean, chargeStationCitiesdf, on = "Unique City", how="inner")
uscitiesStations["Unique City"] = uscitiesStations["Unique City"].astype(str)
# merge city landmass data to new data
uscitiesStationsFinal = pd.merge(uscitiesStations, city_landmassColRemoved, on = "Unique City", how="inner")
uscitiesStationsFinal

,city_ascii,state_id,state_name,lat,lng,population,Density(km^2),Unique City,No. of Charge Stations,Land Area (sq km),radius m
0,NEW YORK,NY,New York,40.6943,-73.9249,18713220,10715.0,"NEW YORK, NY",321,8936,53333
1,LOS ANGELES,CA,California,34.1139,-118.4068,12750807,3276.0,"LOS ANGELES, CA",759,4496,37830
2,CHICAGO,IL,Illinois,41.8373,-87.6862,8604203,4574.0,"CHICAGO, IL",164,6326,44873
3,MIAMI,FL,Florida,25.7839,-80.2102,6445545,5019.0,"MIAMI, FL",147,3208,31955
4,DALLAS,TX,Texas,32.7936,-96.7662,5743938,1526.0,"DALLAS, TX",134,4607,38294
...,...,...,...,...,...,...,...,...,...,...,...
449,KAHULUI,HI,Hawaii,20.8715,-156.4603,32312,867.0,"KAHULUI, HI",17,43,3699
450,FORT WALTON BEACH,FL,Florida,30.4249,-86.6194,22521,1136.0,"FORT WALTON BEACH, FL",7,312,9965
451,WESTMINSTER,MD,Maryland,39.5796,-77.0066,18640,1077.0,"WESTMINSTER, MD",4,149,6886
452,LADY LAKE,FL,Florida,28.9241,-81.9299,16020,747.0,"LADY LAKE, FL",2,184,7653


In [12]:
api_key = "XzSNnJtfDK5IpktOiRC0ZE0n5I5jRh2UX6TEcURPglld794gH8y4vifAS6KaIp1LZnQFEq9sln52Soamph5Lk2v_IcztwvhLoohpjd1rXgqtIhk69shA63HRLMP8X3Yx"
headers = {"Authorization": "Bearer %s" % api_key}

In [14]:
uscitiesStationsFinal["radius m"]

0      53333
1      37830
2      44873
3      31955
4      38294
       ...  
449     3699
450     9965
451     6886
452     7653
453    10387
Name: radius m, Length: 454, dtype: int32

In [26]:

valueList = []
valueCount = 0
pageHitCount = 0
searchTerm = "McDonald's"
pageCounter = 1
searchCompleteVar = False
offset = 0
for index, city in uscitiesStationsFinal.iterrows():
    # limit queries to 7 requests
    for x in range (7):
        # sleep to follow API rules
        time.sleep(1)
        # prepare request     
        url = "https://api.yelp.com/v3/businesses/search"
        params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        #params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        # set up exception handling  
        try:
            # make request and turn into json object
            requestObject = requests.get(url, params = params, headers = headers)
            response = json.loads(requestObject.text)
            print("The status code is {}".format(requestObject.status_code) + " for city: " + city["Unique City"])
            print(requestObject.url)
            # For each response, if the business name is the search term, add to value counter
        except:
            print("Invalid results. Moving to next request.")
            #move to next city  
            pageHitCount = 0
            break
        if requestObject.status_code == 200:
            for x in response["businesses"]:
                if x["name"] == searchTerm:
                    valueCount += 1
                    pageHitCount += 1
                else:
                    #(page 1)
                    #put counter into dataframe
                    print("Found item that isn't the search term.")
                    searchCompleteVar = True
                    pageHitCount = 0
                    break
            if searchCompleteVar == True:
                searchCompleteVar = False
                print(f'Starting new city query.')
                pageHitCount = 0
                break
            pageCounter += 1
            print(f'Moving to page {pageCounter}')
            print(f'found {valueCount} stores in total so far')
            print(f'found {pageHitCount} stores just on this page')            
            offset = pageHitCount + offset
        else:
            print("Server returned error. Moving to next request")
            pageHitCount = 0
            break
    valueList.append(valueCount)
    valueCount = 0    
    offset = 0
    pageCounter = 1




The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=0
Moving to page 2
found 50 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=50
Moving to page 3
found 100 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=100
Moving to page 4
found 150 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=150
Moving to page 5
found 200 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v

The status code is 200 for city: TAMPA, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=TAMPA%2C+FL&limit=50&offset=52
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: DENVER, CO
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=DENVER%2C+CO&limit=50&offset=0
Moving to page 2
found 50 stores in total so far
found 72 stores just on this page
The status code is 200 for city: DENVER, CO
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=DENVER%2C+CO&limit=50&offset=72
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: RIVERSIDE, CA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=RIVERSIDE%2C+CA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: BALTIMORE, MD
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=BALTIMORE%2C+MD&limit=50&of

The status code is 200 for city: PROVIDENCE, RI
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=PROVIDENCE%2C+RI&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: JACKSONVILLE, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=JACKSONVILLE%2C+FL&limit=50&offset=0
Moving to page 2
found 50 stores in total so far
found 64 stores just on this page
The status code is 200 for city: JACKSONVILLE, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=JACKSONVILLE%2C+FL&limit=50&offset=64
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: SALT LAKE CITY, UT
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=SALT+LAKE+CITY%2C+UT&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: NASHVILLE, TN
https://api.yelp.com/v3/businesses/search?term=McDona

The status code is 200 for city: BAKERSFIELD, CA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=BAKERSFIELD%2C+CA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: OGDEN, UT
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=OGDEN%2C+UT&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: BATON ROUGE, LA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=BATON+ROUGE%2C+LA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: AKRON, OH
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=AKRON%2C+OH&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: NEW HAVEN, CT
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+HAVEN%2C+CT&limit=50&offset=0
Fo

The status code is 400 for city: KISSIMMEE, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=KISSIMMEE%2C+FL&limit=50&offset=1267
Server returned error. Moving to next request
The status code is 400 for city: KISSIMMEE, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=KISSIMMEE%2C+FL&limit=50&offset=1267
Server returned error. Moving to next request
The status code is 200 for city: FAYETTEVILLE, AR
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=FAYETTEVILLE%2C+AR&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: PENSACOLA, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=PENSACOLA%2C+FL&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: VICTORVILLE, CA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=VICTORVILLE%2C+CA&limit=50&offset=0
Found item tha

...

The status code is 200 for city: VINELAND, NJ
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=VINELAND%2C+NJ&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: FLORENCE, SC
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=FLORENCE%2C+SC&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: PORTSMOUTH, NH
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=PORTSMOUTH%2C+NH&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: SLIDELL, LA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=SLIDELL%2C+LA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: MISSOULA, MT
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=MISSOULA%2C+MT&limit=50&offset=0
Fo

The status code is 200 for city: ELIZABETHTOWN, KY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=ELIZABETHTOWN%2C+KY&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: TEXARKANA, TX
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=TEXARKANA%2C+TX&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: KANKAKEE, IL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=KANKAKEE%2C+IL&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: WATSONVILLE, CA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=WATSONVILLE%2C+CA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: PORTERVILLE, CA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=PORTERVILLE%2C+C

The status code is 200 for city: LIMA, OH
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=LIMA%2C+OH&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: DECATUR, AL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=DECATUR%2C+AL&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: HANOVER, PA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=HANOVER%2C+PA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: DUBUQUE, IA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=DUBUQUE%2C+IA&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: VICTORIA, TX
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=VICTORIA%2C+TX&limit=50&offset=0
Found item that is

The status code is 200 for city: MIDDLETOWN, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=MIDDLETOWN%2C+NY&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: JEFFERSON CITY, MO
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=JEFFERSON+CITY%2C+MO&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: BECKLEY, WV
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=BECKLEY%2C+WV&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: TITUSVILLE, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=TITUSVILLE%2C+FL&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: CARTERSVILLE, GA
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=CARTERSVILLE%2C

The status code is 200 for city: LADY LAKE, FL
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=LADY+LAKE%2C+FL&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.
The status code is 200 for city: ABERDEEN, MD
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=ABERDEEN%2C+MD&limit=50&offset=0
Found item that isn't the search term.
Starting new city query.


In [27]:
valueList

[258,
 195,
 16,
 50,
 142,
 53,
 140,
 57,
 56,
 29,
 84,
 48,
 16,
 50,
 71,
 52,
 72,
 74,
 47,
 45,
 50,
 63,
 75,
 50,
 66,
 78,
 50,
 55,
 50,
 49,
 50,
 73,
 50,
 69,
 50,
 43,
 50,
 14,
 50,
 45,
 49,
 44,
 48,
 50,
 37,
 50,
 58,
 12,
 39,
 9,
 39,
 30,
 39,
 11,
 41,
 42,
 15,
 45,
 28,
 36,
 14,
 41,
 20,
 9,
 24,
 10,
 18,
 34,
 13,
 5,
 36,
 27,
 13,
 23,
 39,
 13,
 29,
 12,
 5,
 9,
 11,
 18,
 32,
 31,
 18,
 22,
 7,
 26,
 21,
 18,
 25,
 28,
 2,
 8,
 4,
 19,
 21,
 15,
 10,
 25,
 8,
 16,
 14,
 13,
 17,
 50,
 12,
 20,
 6,
 12,
 21,
 17,
 18,
 18,
 3,
 17,
 15,
 7,
 15,
 12,
 8,
 9,
 25,
 6,
 13,
 11,
 9,
 14,
 8,
 13,
 16,
 13,
 15,
 19,
 17,
 13,
 9,
 11,
 18,
 13,
 9,
 9,
 13,
 14,
 12,
 28,
 14,
 41,
 12,
 13,
 15,
 12,
 18,
 16,
 11,
 6,
 6,
 9,
 12,
 12,
 11,
 9,
 6,
 9,
 8,
 13,
 27,
 10,
 5,
 10,
 13,
 11,
 6,
 5,
 10,
 9,
 3,
 6,
 9,
 9,
 6,
 7,
 8,
 7,
 8,
 10,
 10,
 7,
 11,
 21,
 10,
 14,
 5,
 4,
 6,
 8,
 6,
 6,
 11,
 12,
 5,
 6,
 9,
 13,
 9,
 11,
 7,
 9,
 7,
 4,
 5

In [28]:
len(valueList)

454

In [29]:
uscitiesStationsFinal[f'No. of {searchTerm}'] = valueList

In [31]:
uscitiesStationsFinal.to_csv("uscitiesStationFinal.csv")

In [17]:
# NOTE: This code currently will use multiple if statements to decide to request the next page
valueList = []
valueCount = 0
pageHitCount = 0
searchTerm = "McDonald's"

for index, city in uscitiesStationsFinal.head(8).iterrows():
    # sleep to follow API rules
    time.sleep(1)
    # prepare request     
    url = "https://api.yelp.com/v3/businesses/search"
    offset = 0
    params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
    # set up exception handling  
    try:
        # make request and turn into json object
        requestObjectpg1 = requests.get(url, params = params, headers = headers)
        response1 = json.loads(requestObjectpg1.text)
        print("The status code is {}".format(requestObjectpg1.status_code) + " for city: " + city["Unique City"])
        print(requestObjectpg1.url)
        # For each response, if the business name is the search term, add to value counter
    except:
        print("Invalid results. Moving to next request pg1.")
        #move to next city  
    for x in response1["businesses"]:
        if x["name"] == searchTerm:
            valueCount += 1
            pageHitCount += 1
        else:
            #(page 1)
            #put counter into dataframe
            print("First store not found.")
            break
    print(f'found {valueCount} stores in total so far')
    print(f'found {pageHitCount} stores just on this page')
    #if there are more than 50 responses and you still found the search term as 
    #the last response, move on to next page
    if valueCount == 50:
        offset = pageHitCount + offset
        pageHitCount = 0
        url = "https://api.yelp.com/v3/businesses/search"
        params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        # set up exception handling  
        try:
            # make request and turn into json object
            time.sleep(1)
            requestObjectpg2 = requests.get(url, params = params, headers = headers)
            response2 = json.loads(requestObjectpg2.text)
            print("The status code is {}".format(requestObjectpg2.status_code) + " for city: " + city["Unique City"])
            print(requestObjectpg2.url)
            # For each response, if the business name is the search term, add to value counter
        except:
            #(page 1)
            print("Invalid results. Moving to next request pg1.")
            #move to next city  
        for x in response2["businesses"]:
            if x["name"] == searchTerm:
                valueCount += 1
                pageHitCount += 1
            else:
                #(page 1)
                #put counter into dataframe
                print("First store not found.")
                break
        print(f'found {valueCount} stores in total so far')
        print(f'found {pageHitCount} stores just on this page')
    #if there are more than 50 responses and you still found the search term as 
    #the last response, move on to next page                
    if valueCount == 100:
        offset = pageHitCount + offset
        pageHitCount = 0
        url = "https://api.yelp.com/v3/businesses/search"
        params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        # set up exception handling  
        try:
            # make request and turn into json object
            time.sleep(1)
            requestObjectpg3 = requests.get(url, params = params, headers = headers)
            response3 = json.loads(requestObjectpg3.text)
            print("The status code is {}".format(requestObjectpg3.status_code) + " for city: " + city["Unique City"])
            print(requestObjectpg3.url)
            # For each response, if the business name is the search term, add to value counter
        except:
            #(page 1)
            print("Invalid results. Moving to next request pg1.")
            #move to next city  
        for x in response3["businesses"]:
            if x["name"] == searchTerm:
                valueCount += 1
                pageHitCount += 1
            else:
                #(page 1)
                #put counter into dataframe
                print("First store not found.")
                break
        print(f'found {valueCount} stores in total so far')
        print(f'found {pageHitCount} stores just on this page')
                #move to next city
    #if there are more than 50 responses and you still found the search term as              
    #the last response, move on to page 2
    if valueCount == 150:
        offset = pageHitCount + offset
        pageHitCount = 0
        url = "https://api.yelp.com/v3/businesses/search"
        params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        # set up exception handling  
        try:
            # make request and turn into json object
            time.sleep(1)
            requestObjectpg4 = requests.get(url, params = params, headers = headers)
            response4 = json.loads(requestObjectpg4.text)
            print("The status code is {}".format(requestObjectpg4.status_code) + " for city: " + city["Unique City"])
            print(requestObjectpg4.url)
            # For each response, if the business name is the search term, add to value counter
        except:
            #(page 1)
            print("Invalid results. Moving to next request pg1.")
            #move to next city  
        for x in response4["businesses"]:
            if x["name"] == searchTerm:
                valueCount += 1
                pageHitCount += 1
            else:
                #(page 1)
                #put counter into dataframe
                print("First store not found.")
                break
        print(f'found {valueCount} stores in total so far')
        print(f'found {pageHitCount} stores just on this page')
    #if there are more than 50 responses and you still found the search term as               
    #the last response, move on to page 2
    if valueCount == 200:
        offset = pageHitCount + offset
        pageHitCount = 0
        url = "https://api.yelp.com/v3/businesses/search"
        params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        # set up exception handling  
        try:
            # make request and turn into json object
            time.sleep(1)
            requestObjectpg5 = requests.get(url, params = params, headers = headers)
            response5 = json.loads(requestObjectpg5.text)
            print("The status code is {}".format(requestObjectpg5.status_code) + " for city: " + city["Unique City"])
            print(requestObjectpg5.url)
            # For each response, if the business name is the search term, add to value counter
        except:
            #(page 1)
            print("Invalid results. Moving to next request pg1.")
            #move to next city  
        for x in response5["businesses"]:
            if x["name"] == searchTerm:
                valueCount += 1
                pageHitCount += 1
            else:
                #(page 1)
                #put counter into dataframe
                print("First store not found.")
                break
        print(f'found {valueCount} stores in total so far')
        print(f'found {pageHitCount} stores just on this page')
    if valueCount == 250:
        offset = pageHitCount + offset
        pageHitCount = 0
        url = "https://api.yelp.com/v3/businesses/search"
        params = {"term": searchTerm, "location":city["Unique City"], "limit":50, "offset": offset}
        # set up exception handling  
        try:
            # make request and turn into json object
            time.sleep(1)
            requestObjectpg6 = requests.get(url, params = params, headers = headers)
            response6 = json.loads(requestObjectpg6.text)
            print("The status code is {}".format(requestObjectpg6.status_code) + " for city: " + city["Unique City"])
            print(requestObjectpg6.url)
            # For each response, if the business name is the search term, add to value counter
        except:
            #(page 1)
            print("Invalid results. Moving to next request pg1.")
            #move to next city  
        for x in response6["businesses"]:
            if x["name"] == searchTerm:
                valueCount += 1
                print(f'found {valueCount} stores in total so far')
                pageHitCount += 1
                print(f'found {pageHitCount} stores just on this page')
            else:
                #(page 1)
                #put counter into dataframe
                print("First store not found.")
                break
        print(f'found {valueCount} stores in total so far')
        print(f'found {pageHitCount} stores just on this page')
    valueList.append(valueCount)
    valueCount = 0
    pageHitCount = 0    



The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=0
found 50 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=50
found 100 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=100
found 150 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=50&offset=150
found 200 stores in total so far
found 50 stores just on this page
The status code is 200 for city: NEW YORK, NY
https://api.yelp.com/v3/businesses/search?term=McDonald%27s&location=NEW+YORK%2C+NY&limit=

ValueError: Length of values does not match length of index

In [ ]:
uscitiesStationsFinal[f'No. of {searchTerm}'] = valueList
uscitiesStationFinal.to_csv("uscitiesStationFinal.csv")

In [ ]:

'''        

    pageHitCount = len(response1["businesses"])
    if pageHitCount == 50:
        # prepare request for page 2
        offset = pageHitCount + offset
        url = "https://api.yelp.com/v3/businesses/search"
        print (offset)
        params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
        try:
            # make request and turn into json object
            time.sleep(4)
            requestObjectpg2 = requests.get(url, params = params, headers = headers)
            response2 = json.loads(requestObjectpg2.text)
            print("The status code is {}".format(requestObjectp2.status_code) + " for city: " + city["Unique City"] + ", page 2.")
            print(requestObjectpg2.url)
            # For each response, if the business name is "mcdonald's", add to value counter
            for y in response2["businesses"]:
                if y["name"] == "McDonald's":
                    pageHitCount += 1
                    #if there are more than 50 responses and you still found a mcdonald's as 
                    #the last response, move on to next page
                    if pageHitCount == 50:
                        offset = pageHitCount + offset
                        print (offset)
                        params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
                        try:
                            # make request and turn into json object
                            time.sleep(2)
                            requestObjectpg3 = requests.get(url, params = params, headers = headers)
                            response3 = json.loads(requestObjectpg3.text)
                            print("The status code is {}".format(requestObjectpg2.status_code) + " for city: " + city["Unique City"] + ", page 3.")
                            print(requestObjectpg3.url)
                            # For each response, if the business name is "mcdonald's", add to value counter
                            for z in response3["businesses"]:
                                if z["name"] == "McDonald's":
                                    valueCount += 1
                                else:
                                    #(page 3)
                                    #put counter into dataframe
                                    valueList.append(valueCount)
                                    print("Moving to next city.")
                                    break
                                    #move to next city
                        except:
                            #(page 3)
                            print("Invalid results. Moving to next request pg3.")
                            #move to next city    
                    else:
                        #(page 2)
                        #put counter into dataframe
                        valueList.append(valueCount)
                        print("Moving to next city.")
                        break
                        #move to next city
                else:
                        #(page 2)
                        #put counter into dataframe
                        valueList.append(valueCount)
                        print("Moving to next city.")
                        break
                        #move to next city
        except:
                #(page 2)
                print("Invalid results. Moving to next request pg2.")
                #move to next city                    
    else:
        #(page 1)
        #put counter into dataframe
        valueList.append(valueCount)
        print("Moving to next city.")
        break
        #move to next city  
                    
else:
    #(page 1)
    #put counter into dataframe
    valueList.append(valueCount)
    print("Moving to next city.")
    break
    #move to next city                    
except:
    #(page 1)
    print("Invalid results. Moving to next request pg1.")
    #move to next city 
'''

In [ ]:
valueList

In [ ]:
valueList = []
valueList

In [ ]:
pprint(response1)

In [ ]:
pprint(response2)

In [ ]:
pprint(response3)

In [ ]:
pprint(response4)

# ADD IN for each city  uscitiesStations

valueIndex = []
valueCount = 0
currentResult = 0
responseCount = 0
pageHitCount = 0

for index, city in uscitiesStations.iterrows():
    # sleep to follow API rules
    time.sleep(5)
    # prepare request
    url = "https://api.yelp.com/v3/businesses/search"
    offset = 0
    params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
    # set up exception handling  
    try:
        # make request and turn into json object
        requestObjectpg1 = requests.get(url, params = params, headers = headers)
        response1 = json.loads(requestObjectpg1.text)
        print("The status code is {}".format(requestObjectpg1.status_code) + " for city: " + city["Unique City"])
        print(requestObjectpg1.url)
        # For each response, if the business name is "mcdonald's", add to value counter
        for x in response1["businesses"]:
            if x["name"] == "McDonald's":
                valueCount += 1
                pageHitCount += 1
            else:
                #(page 1)
                #put counter into dataframe
                valueIndex.append(valueCount)
                print("First store not found.")
                break
                #move to next city
    #if there are more than 50 responses and you still found a mcdonald's as                 
    #the last response, move on to page 2
    except:
    
    pageHitCount = len(response1["businesses"])
    if pageHitCount == 50:
        # prepare request for page 2
        offset = pageHitCount + offset
        url = "https://api.yelp.com/v3/businesses/search"
        print (offset)
        params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
        try:
            # make request and turn into json object
            time.sleep(4)
            requestObjectpg2 = requests.get(url, params = params, headers = headers)
            response2 = json.loads(requestObjectpg2.text)
            print("The status code is {}".format(requestObjectp2.status_code) + " for city: " + city["Unique City"] + ", page 2.")
            print(requestObjectpg2.url)
            # For each response, if the business name is "mcdonald's", add to value counter
            for y in response2["businesses"]:
                if y["name"] == "McDonald's":
                    pageHitCount += 1
                    #if there are more than 50 responses and you still found a mcdonald's as 
                    #the last response, move on to next page
                    if pageHitCount == 50:
                        offset = pageHitCount + offset
                        print (offset)
                        params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
                        try:
                            # make request and turn into json object
                            time.sleep(2)
                            requestObjectpg3 = requests.get(url, params = params, headers = headers)
                            response3 = json.loads(requestObjectpg3.text)
                            print("The status code is {}".format(requestObjectpg2.status_code) + " for city: " + city["Unique City"] + ", page 3.")
                            print(requestObjectpg3.url)
                            # For each response, if the business name is "mcdonald's", add to value counter
                            for z in response3["businesses"]:
                                if z["name"] == "McDonald's":
                                    valueCount += 1
                                else:
                                    #(page 3)
                                    #put counter into dataframe
                                    valueIndex.append(valueCount)
                                    print("Moving to next city.")
                                    break
                                    #move to next city
                        except:
                            #(page 3)
                            print("Invalid results. Moving to next request pg3.")
                            #move to next city    
                    else:
                        #(page 2)
                        #put counter into dataframe
                        valueIndex.append(valueCount)
                        print("Moving to next city.")
                        break
                        #move to next city
                else:
                        #(page 2)
                        #put counter into dataframe
                        valueIndex.append(valueCount)
                        print("Moving to next city.")
                        break
                        #move to next city
        except:
                #(page 2)
                print("Invalid results. Moving to next request pg2.")
                #move to next city                    
    else:
        #(page 1)
        #put counter into dataframe
        valueIndex.append(valueCount)
        print("Moving to next city.")
        break
        #move to next city  
                    
else:
    #(page 1)
    #put counter into dataframe
    valueIndex.append(valueCount)
    print("Moving to next city.")
    break
    #move to next city                    
except:
    #(page 1)
    print("Invalid results. Moving to next request pg1.")
    #move to next city                      

# ADD IN for each city  uscitiesStations

valueIndex = [1,2,3]
valueCount = 0
currentResult = 0
responseCount = 0

for index, city in uscitiesStations.iterrows():
    # sleep to follow API rules
    time.sleep(5)
    # prepare request
    url = "https://api.yelp.com/v3/businesses/search"
    offset = 0
    params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
    # set up exception handling  
    try:
        # make request and turn into json object
        requestObjectpg1 = requests.get(url, params = params, headers = headers)
        response1 = json.loads(requestObjectpg1.text)
        print("The status code is {}".format(requestObjectpg1.status_code) + " for city: " + city["Unique City"])
        print(requestObjectpg1.url)
        # For each response, if the business name is "mcdonald's", add to value counter
        for x in response1["businesses"]:
            if x["name"] == "McDonald's":
                valueCount += 1
                #if there are more than 50 responses and you still found a mcdonald's as                 
                #the last response, move on to page 2
                responseCount = len(response1["businesses"])
                if responseCount == 50:
                    # prepare request for page 2
                    offset = responseCount + offset
                    url = "https://api.yelp.com/v3/businesses/search"
                    print (offset)
                    params = {"term": "McDonald's", "location":city["SAN DIEGO, CA"], "limit":50, "offset": offset}
                    try:
                        # make request and turn into json object
                        time.sleep(4)
                        requestObjectpg2 = requests.get(url, params = params, headers = headers)
                        response2 = json.loads(requestObjectpg2.text)
                        print("The status code is {}".format(requestObjectp2.status_code) + " for city: " + city + ", page 2.")
                        print(requestObjectpg2.url)
                        # For each response, if the business name is "mcdonald's", add to value counter
                        for y in response2["businesses"]:
                            if y["name"] == "McDonald's":
                                valueCount += 1
                                #if there are more than 50 responses and you still found a mcdonald's as 
                                #the last response, move on to next page
                                if responseCount == 50:
                                    offset = responseCount + offset
                                    print (offset)
                                    params = {"term": "McDonald's", "location":city["Unique City"], "limit":50, "offset": offset}
                                    try:
                                        # make request and turn into json object
                                        time.sleep(2)
                                        requestObjectpg3 = requests.get(url, params = params, headers = headers)
                                        response3 = json.loads(requestObjectpg3.text)
                                        print("The status code is {}".format(requestObjectpg2.status_code) + " for city: " + city + ", page 3.")
                                        print(requestObjectpg3.url)
                                        # For each response, if the business name is "mcdonald's", add to value counter
                                        for z in response3["businesses"]:
                                            if z["name"] == "McDonald's":
                                                valueCount += 1
                                            else:
                                                #(page 3)
                                                #put counter into dataframe
                                                valueIndex.append(valueCount)
                                                print("Moving to next city.")
                                                break
                                                #move to next city
                                    except:
                                        #(page 3)
                                        print("Invalid results. Moving to next request pg3.")
                                        #move to next city    
                                else:
                                    #(page 2)
                                    #put counter into dataframe
                                    valueIndex.append(valueCount)
                                    print("Moving to next city.")
                                    break
                                    #move to next city
                            else:
                                    #(page 2)
                                    #put counter into dataframe
                                    valueIndex.append(valueCount)
                                    print("Moving to next city.")
                                    break
                                    #move to next city
                    except:
                            #(page 2)
                            print("Invalid results. Moving to next request pg2.")
                            #move to next city                    
                else:
                    #(page 1)
                    #put counter into dataframe
                    valueIndex.append(valueCount)
                    print("Moving to next city.")
                    break
                    #move to next city  
                    
            else:
                #(page 1)
                #put counter into dataframe
                valueIndex.append(valueCount)
                print("Moving to next city.")
                break
                #move to next city                    
    except:
        #(page 1)
        print("Invalid results. Moving to next request pg1.")
        #move to next city                      